# Парсинг основных данных по отелям с сайта tophotels.ru

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
from IPython.display import clear_output
from tqdm import tqdm
from fake_useragent import UserAgent

In [4]:
import numpy as np

In [5]:
import matplotlib.pyplot as plt

## Сначала достаем список отелей в заданной местности. В работе ограничимся Санкт-Петербургом, Крымом и Сочи (два черноморских курорта и северный город)

### Интересующие данные: название, район, рейтинг, кол-во отзывов, ссылка на подробное описание

In [6]:
root_link='https://tophotels.ru'

## Следующая сслыка задана уже с учетом региона, по которому парсим отели

In [8]:
#if len(sys.argv)>1: # задел для запуска из командной строки; корректная работа не гарантирована
    #main_link=sys.argv[1]
#else:
    #main_link="https://tophotels.ru/hotels/1/669"
main_link="https://tophotels.ru/hotels/1/669" # Санкт-Петербург

In [9]:
#hotel_number=get_hotel_number()
#print("Hotels total: ", hotel_number)
hotel_number=1702

In [25]:
first_page=1
last_page=int(np.ceil(hotel_number/50))
last_page
#print("Pages total: ", last_page)

35

In [11]:
def page_link(page, link=main_link): # ссылка на страницу соответствующей выдачи поиска
    return link+'?page='+str(page)

In [12]:
def get_hotels(link): # для удобства - лист из html-контентов на странице, которые, собственно, уже содержат
    response = requests.get(link, headers={'User-Agent': UserAgent().msie}) # остальные данные
    soup = BeautifulSoup(response.content, "html.parser")
    hotels=soup.findAll("div", attrs={'class':'catalogs-item'})
    return None if not hotels else hotels

In [13]:
def get_score(hotel): # рейтинг отеля
    score=hotel.find("span", attrs={'class':'page-ttls-hotel-rating'})
    return None if not score else score.text.strip()

In [14]:
def get_area(hotel): # район, где расположен отель
    area=hotel.find("a", attrs={'class':'catalogs-gray'})
    return None if not area else area.text.strip()

In [15]:
def get_title(hotel): # название отеля (может быть неуникально!)
    title=hotel.find('a', attrs={'class':'catalogs-ttl-a'})
    return None if not title else title.text.strip()

In [16]:
def get_ref(hotel, r_link=root_link): # ссылка на подробное описание отеля и тексты отзывов, уникальна для каждого отеля
    rev=hotel.findAll('a', attrs={'class':'catalogs-per-a'})
    if len(rev)==0:
        return None
    if len(rev)==1:
        ref=r_link+rev[0]['href']
    else:
        ref=r_link+rev[1]['href']
    return ref

In [17]:
def get_nreviews(hotel): # число отзывов для каждого отеля
    rev=hotel.findAll('a', attrs={'class':'catalogs-per-a'})
    if len(rev)==0: # как потом выяснилось, не всегда точно совпадает
        return None #  с реальным числом отзывов (но близко) - реальное число отзывов может быть больше.
    if len(rev)==1:
        n=rev[0].text.strip()
    else:
        n=rev[1].text.strip()
    return n

In [18]:
def get_line(hotel): # делаем словарик из данных, упомянутых выше, чтоб добавить потом в общий датафрейм
    title=get_title(hotel)
    area=get_area(hotel)
    score=get_score(hotel)
    nrev=get_nreviews(hotel)
    ref=get_ref(hotel)
    return {'title':title,'area':area,'rating':score,'n_reviews':nrev,'reference':ref}

In [26]:
col_names=['title','area','rating','n_reviews','reference']

In [27]:
df=pd.DataFrame(columns=col_names)

In [28]:
t1=tqdm(range(first_page,last_page+1), leave=False) # добываем данные в цикле и формируем датафрейм
for i in t1:
    hotels = get_hotels(page_link(i))
    time.sleep(0.3)
    t2=tqdm(hotels, leave=False)
    for hotel in t2:
        df=df.append(get_line(hotel),ignore_index=True)
    t2.close()
    clear_output()
t1.close()
clear_output()

In [29]:
len(df)

1702

In [30]:
df.isna().sum()

title          0
area           0
rating       957
n_reviews    109
reference    109
dtype: int64

## Итак, всего 1702 отеля, у примерно половины из которых нет рейтинга

In [31]:
df.to_csv("data_raw.csv",header=df.columns,index=False)

### Сохраняем в сыром виде, затем небольшая предварительная чистка

In [32]:
df.rating=df.rating.astype('float')

In [33]:
#plt.hist(Crimea.rating.dropna())

In [34]:
df.dropna(axis=0,subset=['n_reviews'],inplace=True)

In [35]:
#Crimea.head()

In [36]:
#a=Crimea.loc[Crimea.n_reviews.apply(lambda x: x[1:4])=='ото',"n_reviews"]

In [37]:
#a.head()

In [38]:
df.loc[df.n_reviews.apply(lambda x: x[1:4])=='ото',"n_reviews"]="Отзывы - 0"

In [39]:
df.to_csv("data_raw_stage2.csv",header=df.columns,index=False)

In [40]:
df.n_reviews=df.n_reviews.apply(lambda x: x[9:].strip()).astype("int32")

In [41]:
#Crimea.describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.9,0.99])

In [42]:
df.to_csv("data_raw_stage3.csv",header=df.columns,index=False)

In [43]:
index_to_drop=df.loc[df["n_reviews"]==0,:].index

In [44]:
df.drop(index=index_to_drop, axis=0, inplace=True)

In [47]:
len(df)

893

In [48]:
df.rating.fillna(0,inplace=True)

In [49]:
#Crimea.describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.9,0.99])

### Сохраняем очищенные данные

In [50]:
df.to_csv("hotel_stats_clean.csv",header=df.columns,index=False)